Sources:
- lectures by Michael Granitzer (michael.granitzer@uni-passau.de)
- https://medium.com/machine-learning-guy/an-introduction-to-decision-tree-learning-id3-algorithm-54c74eb2ad55
-https://towardsdatascience.com/decision-trees-for-classification-id3-algorithm-explained-89df76e72df1

In [5]:
import pandas as pd
import numpy as np

# Goal: Implement the ID3-Algorithm

Experiments are conducted on the [UCI CAR Dataset](http://archive.ics.uci.edu/ml/machine-learning-databases/car/).

We will cover
  
  * Data Setup
  * Impurity functions and impurity reduction
  * ID3 Algorithm for learning a decision tree
  * Classifying unseen examples
  * Evaluation using cross validation

### Pseudocode¶

`ID3 (Examples, Target_Attribute, Attributes)
    Create a root node for the tree
    If all examples are positive, Return the single-node tree Root, with label = +.
    If all examples are negative, Return the single-node tree Root, with label = -.
    If number of predicting attributes is empty, then Return the single node tree Root,
    with label = most common value of the target attribute in the examples.
    Otherwise Begin
        A ← The Attribute that best classifies examples.
        Decision Tree attribute for Root = A.
        For each possible value, vi, of A,
            Add a new tree branch below Root, corresponding to the test A = vi.
            Let Examples(vi) be the subset of examples that have the value vi for A
            If Examples(vi) is empty
                Then below this new branch add a leaf node with label = most common target value in the examples
            Else below this new branch add the subtree ID3 (Examples(vi), Target_Attribute, Attributes – {A})
    End
    Return Root`
    


## Data Setup

### Download and import the data

In [1]:
#shell scripts for downloading the data and placing it in a corresponding directory
!mkdir CAR 
!curl -o CAR/data "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
!curl -o CAR/description "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.names"
#download the description and display it here.
!cat CAR/description

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 51867  100 51867    0     0  59892      0 --:--:-- --:--:-- --:--:-- 59892
100 51867  100 51867    0     0  59754      0 --:--:-- --:--:-- --:--:-- 59823
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3097  100  3097    0     0   6703      0 --:--:-- --:--:-- --:--:--  6718
"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [6]:
col_names = ['price_buy', 'price_main', 'n_doors', 'n_persons', 'lug_boot', 'safety', 'recommendation']
df = pd.read_csv("./CAR/data", header=None, names=col_names)


# All attributes are categorical - a mix of strings and integers.
# We simply map the categorical values of each attribute to a set of distinct integers
ai2an_map = col_names
ai2aiv2aivn_map = []
enc_cols = []
for col in df.columns:
    df[col] = df[col].astype('category')
    a = np.array(df[col].cat.codes.values).reshape((-1,1))
    enc_cols.append(a)
    ai2aiv2aivn_map.append(list(df[col].cat.categories.values))

    
    
# Get the data as numpy 2d-matrix (n_samples, n_features)
dataset = np.hstack(enc_cols)
df_new=pd.DataFrame(dataset, index=None, columns=col_names)
print(df_new)
X, y = dataset[:,:6], dataset[:,6]
print(X.shape, y.shape)

      price_buy  price_main  n_doors  n_persons  lug_boot  safety  \
0             3           3        0          0         2       1   
1             3           3        0          0         2       2   
2             3           3        0          0         2       0   
3             3           3        0          0         1       1   
4             3           3        0          0         1       2   
...         ...         ...      ...        ...       ...     ...   
1723          1           1        3          2         1       2   
1724          1           1        3          2         1       0   
1725          1           1        3          2         0       1   
1726          1           1        3          2         0       2   
1727          1           1        3          2         0       0   

      recommendation  
0                  2  
1                  2  
2                  2  
3                  2  
4                  2  
...              ...  
1723      

## Model Implementation

In [7]:
class DecisionTreeID3():
    def __init__(self, target_name='recommendation'):
        """
        tsrget_name - name for target column
        first_gini - list for counting gini
        """
        self.target_name=target_name
        self.first_gini=[] 
        self.result=0
        
    def _gini(self, data, feature_name,category,target_name):
        """
        Description:
        Function for calculating gini impurity(G_i = 1- sum(P_i_k)^2).

        Args:
        data - dataset of data(pandas.DataFrame)
        feature_name - list of features 
        category - list of categories (what can be inside every feature)
        target_name - target name. 

        Returns:
        gini-impurity 
        """
        #calculate the length for category in features
        common_length=len(data[data[feature_name]==category])    
        def P_i_K(target, feature_name):
            #calculate probability for every variants in target column for avary category in feature
            return len(data[(data[feature_name]==category) & (data[target_name]==target)])/common_length
        #impurity of every category in feature 
        gini_impurity=1-sum(P_i_K(target, feature_name)**2 for target in set(data[target_name])) 
        return gini_impurity


    def _total_gini(self, data, feature_name, target_name):
        """
        Description:
        Function for calculation total gini impurity (G=sum(Gini_i*P_k_a))

        Args:
        data - dataset of data(pandas.DataFrame)
        feature_name - list of features 
        target_name - target name. 

        Returns:
        Total gini  - information gain
        """
        def P_k_a(category, feature_name):
            #probability for every category in whole data
            return len(data[data[feature_name]==category])/len(data)
        #for every category in feature
        for category in set(data[feature_name]):
            #calculate information gain for every feature
            gini_value=self._gini(data, feature_name, category, self.target_name)
            P_k_a_value=P_k_a(category, feature_name)
            self.result+=gini_value*P_k_a_value
        return self.result
  

    def _result(self, data):
        """
        Description:
        Function for collecting information gains for all features for every node


        Args:
        data - dataset of data(pandas.DataFrame)

        Returns:
        First_gini - list of result for total gini impurity
        """
        #feature names - all feature before last column
        feature_names=data.keys()[:-1]
        for feature_name in feature_names: 
            self.first_gini.append(self._total_gini(data, feature_name, self.target_name))
        return self.first_gini
    
    
    def _buildTree(self, data, tree=None):
        """
        Description:
        Function for building tree
        
        Args: 
        data - dataset of data(pandas.DataFrame)
        tree  - vocabulary with tree
        
        Returns:
        tree"""
        #features which we will use for predictions, future nodes
        feature_names=data.keys()[:-1]
        #dictionary with features+information gain for them(can be via entropy or gini)
        voc=dict(zip(feature_names,self._result(data)))
        
        #if we use gini then we will choose minimum one
        node=min(voc, key=voc.get)
        
        if tree is None:
            tree={}
            tree[node]={}

        attributes=np.unique(data[node])

        #just go deep into the tree, for every nodes we cheking information gain 
        for value in attributes:
            df_new_2=data[data[node]==value]
            clValue,counts=np.unique(df_new_2[self.target_name], return_counts=True)

            if len(counts)==1:
                tree[node][value]=clValue[0]
            else:
                tree[node][value]=self._buildTree(df_new_2.drop(columns=node))
        
        return tree
    
    
    def _predict(self, inst, tree):
        """
        Description:
        Function for prediction using tree

        Args:
        inst- data for prediction - pandas.Series
        tree- tree from training
        
        Returns:
        prediction
        """
        for nodes in tree.keys():
            value=inst[nodes]
            tree=tree[nodes][value]
            prediction=0
            
            if type(tree) is dict:
                prediction=self._predict(inst,tree)
            else:
                prediction=tree
                break;

        return prediction

In [8]:
model=DecisionTreeID3()

In [13]:
tree_for_model=model._buildTree(df_new)

### Prediction

In [9]:
data_for_test={'price_buy':1, 'price_main':2, 'n_doors':3, 'n_persons':2, 'lug_boot':2, 'safety':0}

In [10]:
inst=pd.Series(data_for_test)

In [12]:
#prediction which car I should buy
print("The car is %s"%model._predict(inst,tree_for_model))

The car is 1
